# 🚀 RAG SaaS Platform - Python Versiyonu (Colab Kurulumu)

**100% Python - FastAPI + Streamlit**

**Basit Adımlar:**
1. Projeyi yükle
2. Bağımlılıkları kur
3. API key ayarla
4. Servisleri başlat
5. Public URL al

---


## 1️⃣ Projeyi GitHub'dan Yükle


In [ ]:
!git clone https://github.com/mmcanpolat/rag_nlp_chatbotplatform.git
%cd rag_nlp_chatbotplatform
print("✅ Proje yüklendi!")


## 2️⃣ Tüm Bağımlılıkları Kur

Python paketleri kuruluyor (5-10 dakika sürebilir).


In [ ]:
print("📦 Python paketleri kuruluyor...")
!pip install -q -r python_services/requirements.txt

print("\n✅ Tüm bağımlılıklar kuruldu!")


## 3️⃣ API Key'i Ayarla

**İki Yöntem:**

**🔑 Yöntem A: Colab Secrets (Önerilen - En Güvenli)**
1. Sol menüden 🔑 (Key) ikonuna tıklayın
2. "Add new secret" → Name: `OPENAI_API_KEY`, Value: API key'iniz
3. Aşağıdaki kodu çalıştırın

**⌨️ Yöntem B: Gizli Input (Alternatif)**
Aşağıdaki kodu çalıştırın, görünmez şekilde giriş yapın


In [ ]:
# Yöntem A: Colab Secrets kullan (önerilen)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        raise KeyError("Secret bulunamadı")
    print("✅ API key Colab Secrets'tan alındı")
except (KeyError, ModuleNotFoundError):
    # Yöntem B: Gizli input (fallback)
    from getpass import getpass
    print("⚠️ Colab Secrets bulunamadı. Gizli input kullanılıyor...")
    print("💡 İpucu: Sol menüden 🔑 ikonuna tıklayarak Colab Secrets kullanabilirsiniz")
    OPENAI_API_KEY = getpass("API Key (görünmez): ")

if not OPENAI_API_KEY or OPENAI_API_KEY == "":
    print("❌ API Key boş! Lütfen tekrar deneyin.")
    raise ValueError("API Key gerekli!")
else:
    # .env dosyalarını oluştur
    import os
    os.makedirs('backend_fastapi', exist_ok=True)
    os.makedirs('python_services', exist_ok=True)
    
    with open('backend_fastapi/.env', 'w') as f:
        f.write(f"PORT=3000\nOPENAI_API_KEY={OPENAI_API_KEY}\nREQUEST_TIMEOUT=600000")
    
    with open('python_services/.env', 'w') as f:
        f.write(f"OPENAI_API_KEY={OPENAI_API_KEY}")
    
    # Environment variable olarak da set et
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    
    print("✅ API key güvenli şekilde ayarlandı (.env dosyalarına kaydedildi)")


In [ ]:
import subprocess
import time
import os
import sys

# Eski process'leri durdur
print("🛑 Eski process'leri durduruyorum...")
subprocess.run(['pkill', '-f', 'uvicorn'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', 'streamlit'], stderr=subprocess.DEVNULL)
time.sleep(2)

# Backend başlat (FastAPI)
print("\n🔄 Backend (FastAPI) başlatılıyor...")
backend = subprocess.Popen([
    sys.executable, '-m', 'uvicorn',
    'main:app',
    '--host', '0.0.0.0',
    '--port', '3000'
], 
cwd='backend_fastapi',
stdout=open('/tmp/backend.log', 'w'),
stderr=subprocess.STDOUT,
env={**os.environ, 'OPENAI_API_KEY': OPENAI_API_KEY})
time.sleep(5)

# Frontend başlat (Streamlit)
print("🔄 Frontend (Streamlit) başlatılıyor...")
frontend = subprocess.Popen([
    sys.executable, '-m', 'streamlit', 'run',
    'app.py',
    '--server.port', '8501',
    '--server.address', '0.0.0.0',
    '--server.headless', 'true'
],
cwd='frontend_streamlit',
stdout=open('/tmp/frontend.log', 'w'),
stderr=subprocess.STDOUT,
env={**os.environ, 'API_BASE_URL': 'http://localhost:3000'})
time.sleep(10)

print("\n✅ Servisler başlatıldı!")
print("📍 Backend: http://localhost:3000")
print("📍 Frontend: http://localhost:8501")
print("\n📋 Logları görmek için:")
print("   !tail -20 /tmp/backend.log")
print("   !tail -20 /tmp/frontend.log")


In [ ]:
# localtunnel kur (opsiyonel)
!pip install -q localtunnel

import subprocess
import time
import re

print("🌐 Public URL oluşturuluyor...")

process = subprocess.Popen(['lt', '--port', '8501'], 
                          stdout=subprocess.PIPE, 
                          stderr=subprocess.STDOUT,
                          text=True)

time.sleep(8)

output = process.stdout.read(1000) if process.stdout else ""
print(output)

url_match = re.search(r'https://[a-z0-9-]+\.loca\.lt', output)
if url_match:
    public_url = url_match.group(0)
    print(f"\n✅ PUBLIC URL: {public_url}")
    print(f"📍 Bu URL'yi tarayıcıda açabilirsiniz!")
else:
    print("\n⚠️ Terminal çıktısına bakın, URL orada görünecek")


## ✅ Hazır!

**Giriş Bilgileri:**
- Email: `admin@ragplatform.com`
- Şifre: `Admin123!@#`

**Servis Durumunu Kontrol Et:**


In [ ]:
import requests
import time

time.sleep(5)

print("📊 Servis durumunu kontrol ediyorum...\n")

# Backend kontrolü
try:
    response = requests.get('http://localhost:3000/api/health', timeout=5)
    print(f"✅ Backend: {response.status_code} - Çalışıyor")
except Exception as e:
    print(f"❌ Backend: Yanıt vermiyor - {e}")

# Frontend kontrolü
try:
    response = requests.get('http://localhost:8501', timeout=5)
    print(f"✅ Frontend: {response.status_code} - Çalışıyor")
except Exception as e:
    print(f"❌ Frontend: Yanıt vermiyor - {e}")
    print("   (Hala başlatılıyor olabilir, birkaç saniye bekleyin)")
